# Initializing the SysML v2 API

In [1]:
from __future__ import print_function

import time
import requests
from pprint import pprint
import pandas as pd
import json

host = "http://sysml2-dev.intercax.com:9000"

# Get projects

In [2]:
projects_url = f"{host}/projects" 
projects_response = requests.get(projects_url)

if projects_response.status_code == 200:
    projects = projects_response.json()
    
    df = pd.DataFrame({'Project Name':[], 'Project ID':[]})
    for p in projects:
        df = pd.concat([df, pd.DataFrame({'Project Name':[p['name']], 'Project ID':[p['@id']]})], ignore_index=True)
    display(df)

,Project Name,Project ID
0,13b-Safety and Security Features Element Group...,035a340b-ee4d-451d-9b28-196d100b0469
1,14b-Language Extensions Wed Mar 09 12:20:20 ES...,0567ee31-4238-40b9-989f-20764fcfdf6d
2,Expressions Wed Mar 09 12:29:23 EST 2022,06574af0-fbdf-42c4-8dbb-ee1b0157c106
3,15_19-Materials with Properties Wed Mar 09 12:...,0d6d0f0c-ce74-4fc3-9f4f-f63434103c41
4,12b-Allocation Wed Mar 09 12:19:12 EST 2022,1f6205f3-9503-4245-8c80-27d0731dd22c
5,13b-Safety and Security Features Element Group...,22053307-618c-4a9d-bfbf-fbc4dad2ce19
6,17a-Sequence-Modeling Wed Mar 09 12:21:27 EST ...,2a17590a-d896-42d8-8a61-1a29a93ef67b
7,10c-Fuel Economy Analysis Wed Mar 09 12:17:31 ...,31b4d90d-4f67-4859-a92e-dd5faaf1a071
8,4a-Functional Allocation Wed Mar 09 12:14:16 E...,383c9281-b522-488e-bc3d-7cca6524671a
9,3a-Function-based Behavior-2 Wed Mar 09 12:13:...,38865031-b14f-4eb9-9cfe-4242be523332


# Define a function to recursively iterate over feature members 
## Recursive graph traversal using FeatureMembership relations

In [3]:
def get_member_features(project_id, commit_id, element_id, member_type, indent):
    
    # Fetch the element
    element_url = f"{host}/projects/{project_id}/commits/{commit_id}/elements/{element_id}" 
    response = requests.get(element_url)
    
    element_data = response.json()
    element_name = element_data['name']
    element_id = element_data['@id']
    element_type = element_data ['@type']
    
    if element_type == member_type:
        print(f"{indent} - {element_name} (id = {element_id}, type = {element_type})")
        # Feature memberships
        element_features = element_data['feature']
        if len(element_features) > 0:
            for feature in element_features:
                get_member_features(project_id, commit_id, feature['@id'], member_type, indent + "  ")


# Parts tree navigation

In [4]:
parts_tree_project_id = '80d912f3-5d03-4d89-a2ce-3df5a54f6837'
parts_tree_commit_id = ''

commits_url = f"{host}/projects/{parts_tree_project_id}/commits" 
commits_response = requests.get(commits_url)
if commits_response.status_code == 200:
    commits = commits_response.json()
    pprint(commits)
    parts_tree_commit_id = commits[0]['@id']


[{'@id': 'f0e92484-92fb-4afd-a2b3-7abaebc46ee6',
  '@type': 'Commit',
  'owningProject': {'@id': '80d912f3-5d03-4d89-a2ce-3df5a54f6837'},
  'previousCommit': None,
  'timestamp': '2022-03-09T12:12:25.727951-05:00'}]


## Query to find vehicle1

In [5]:
# Query vehicle1 (PartUsage) in the 
data = {
  '@type':'Query',
  'select': ['name','@id','@type','owner'],
  'where': {
    '@type': 'CompositeConstraint',
    'operator': 'and',
    'constraint': [
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': 'name',
            'value': 'vehicle1'
        },
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': '@type',
            'value': 'PartUsage'
        }
    ]
  }
}

payload = json.dumps(data)

vehicle1_id = ''
query_url = f"{host}/projects/{parts_tree_project_id}/query-results" 

query_response = requests.post(query_url, json=data)

if query_response.status_code == 200:
    query_response_json = query_response.json()
    pprint(query_response_json)
    vehicle1_id = query_response_json[0]['@id']

[{'@id': 'd5156ae4-c9e2-4c23-93e1-0ba56ebe36bf',
  '@type': 'PartUsage',
  'name': 'vehicle1',
  'owner': {'@id': 'bcf497b1-d1e1-43e2-aed5-2822643a7ce9'}}]


## Parts tree for vehicle1 (recursive FeatureMembership relation navigation)

In [6]:
get_member_features(parts_tree_project_id,parts_tree_commit_id,vehicle1_id,'PartUsage'," ")

  - vehicle1 (id = d5156ae4-c9e2-4c23-93e1-0ba56ebe36bf, type = PartUsage)
    - frontAxleAssembly (id = fad78fd1-5519-42c4-be05-34f17a115f89, type = PartUsage)
      - frontAxle (id = 69a3a7f5-f9c7-4829-9d27-e0cc45d7b96b, type = PartUsage)
      - frontWheel (id = c9b3d19d-95ed-4404-b4bb-e79026861c82, type = PartUsage)
    - rearAxleAssembly (id = 828fb44e-ecce-4b47-8139-d68eef254180, type = PartUsage)
      - rearAxle (id = 77e64a83-7637-46ca-b343-3996bd2cdb4d, type = PartUsage)
      - rearWheel (id = 36a1f7e5-ab10-4320-8ee4-cb62c13aeb87, type = PartUsage)


## Query to find vehicle1_c1

In [7]:

data = {
  '@type':'Query',
  'select': ['name','@id','@type','owner'],
  'where': {
    '@type': 'CompositeConstraint',
    'operator': 'and',
    'constraint': [
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': 'name',
            'value': 'vehicle1_c1'
        },
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': '@type',
            'value': 'PartUsage'
        }
    ]
  }
}
payload = json.dumps(data)

vehicle1_c1_id = ''
query_url = f"{host}/projects/{parts_tree_project_id}/query-results" 

query_response = requests.post(query_url, json=data)

if query_response.status_code == 200:
    query_response_json = query_response.json()
    pprint(query_response_json)
    vehicle1_c1_id = query_response_json[0]['@id']

[{'@id': '1ff997c6-7de9-402c-90a3-a9fcc238bb38',
  '@type': 'PartUsage',
  'name': 'vehicle1_c1',
  'owner': {'@id': 'bcf497b1-d1e1-43e2-aed5-2822643a7ce9'}}]


## Parts tree for vehicle1_c1 (recursive FeatureMembership relation navigation)

In [8]:
get_member_features(parts_tree_project_id,parts_tree_commit_id,vehicle1_c1_id,'PartUsage'," ")

  - vehicle1_c1 (id = 1ff997c6-7de9-402c-90a3-a9fcc238bb38, type = PartUsage)
    - frontAxleAssembly_c1 (id = d76ce4de-b94d-4bdc-849b-481133d73b78, type = PartUsage)
      - frontAxle_c1 (id = cb2eda6b-0a5b-4e9a-ad6a-2d44a3054ac5, type = PartUsage)
      - frontWheel_1 (id = 9a578db7-f9c3-4728-be2e-4fd959802ab6, type = PartUsage)
      - frontWheel_2 (id = 74183ff0-f7e7-4dec-89bb-c76d9f9dbfeb, type = PartUsage)
      - frontWheel (id = c9b3d19d-95ed-4404-b4bb-e79026861c82, type = PartUsage)
    - rearAxleAssembly_c1 (id = 8bf04eb1-6203-4acc-8529-eb66b9412812, type = PartUsage)
      - rearAxle_c1 (id = 82df4d3e-2d95-4b3d-bcf8-4d65502aca54, type = PartUsage)
      - rearWheel_1 (id = ec63812a-588c-4f15-9eb9-2035eb7c8464, type = PartUsage)
      - rearWheel_2 (id = e26b6881-5503-462b-aeee-a58a4d02d134, type = PartUsage)
      - rearWheel (id = 36a1f7e5-ab10-4320-8ee4-cb62c13aeb87, type = PartUsage)


# Behavior Tree Navigation

In [9]:
behavior_project_id = '38865031-b14f-4eb9-9cfe-4242be523332'
behavior_project_commit_id = ''

commits_url = f"{host}/projects/{behavior_project_id}/commits" 
commits_response = requests.get(commits_url)
if commits_response.status_code == 200:
    commits = commits_response.json()
    pprint(commits)
    behavior_project_commit_id = commits[0]['@id']

[{'@id': 'cbac34f5-e684-457e-b5a9-11f03960583c',
  '@type': 'Commit',
  'owningProject': {'@id': '38865031-b14f-4eb9-9cfe-4242be523332'},
  'previousCommit': None,
  'timestamp': '2022-03-09T12:13:42.401302-05:00'}]


## Query to find Provide Power behavior (ActionUsage)

In [10]:
data = {
  '@type':'Query',
  'select': ['name','@id','@type','owner'],
  'where': {
    '@type': 'CompositeConstraint',
    'operator': 'and',
    'constraint': [
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': 'name',
            'value': 'provide power'
        },
        {
            '@type': 'PrimitiveConstraint',
            'inverse': False,
            'operator': '=',
            'property': '@type',
            'value': 'ActionUsage'
        }
    ]
  }
}

payload = json.dumps(data)

provide_power_id = ''
query_url = f"{host}/projects/{behavior_project_id}/query-results" 

query_response = requests.post(query_url, json=data)

if query_response.status_code == 200:
    query_response_json = query_response.json()
    pprint(query_response_json)
    provide_power_id = query_response_json[0]['@id']

[{'@id': '8cdb0222-c23b-4a3f-97b7-dbe85d563f24',
  '@type': 'ActionUsage',
  'name': 'provide power',
  'owner': {'@id': '5a2fbc37-abbd-4d15-bc1a-c207692b9d73'}}]


## Behavior tree for vehicle1 (recursive FeatureMembership relation navigation)

In [11]:
get_member_features(behavior_project_id, behavior_project_commit_id, provide_power_id,"ActionUsage"," ")

  - provide power (id = 8cdb0222-c23b-4a3f-97b7-dbe85d563f24, type = ActionUsage)
    - generate torque (id = 658fc445-4ad8-4214-bad2-705e3e89e8a2, type = ActionUsage)
    - amplify torque (id = 669056b2-1c35-43b5-88e2-011decf4c8aa, type = ActionUsage)
    - transfer torque (id = 7d8a580a-8f21-4a28-b449-938d02b4f528, type = ActionUsage)
    - distribute torque (id = 0948ab11-2856-45be-b70a-44504759ebef, type = ActionUsage)
    - start (id = 6a7609c8-85fa-4df5-bb92-23e3b14907ed, type = ActionUsage)


# Requirement tree for vehicle

In [12]:
vehicle_project = '4c5b2a9a-5bd1-4fd8-8c2f-548e0a8e5481'
vehicle_commit = 'a6316295-5a01-4176-9a55-d01f058d0668'
vehicle_spec = 'e405e7de-a408-4aa1-898e-6d2b6eee03a7'
host = "http://sysml2-sst.intercax.com:9000"
get_member_features(vehicle_project, vehicle_commit, vehicle_spec,"RequirementUsage"," ")

  - vehicleSpecification (id = e405e7de-a408-4aa1-898e-6d2b6eee03a7, type = RequirementUsage)
    - vehicleMassRequirement (id = 9da27f21-7345-4ce4-8257-c90422ad5403, type = RequirementUsage)
    - vehicleFuelEconomyRequirements (id = 94854c0a-e4ae-4136-b49c-74e05f6dea93, type = RequirementUsage)
      - cityFuelEconomyRequirement (id = 9d8069c9-7a4f-4830-8efa-681eaad10bbc, type = RequirementUsage)
      - highwayFuelEconomyRequirement (id = dfb84e6f-b940-44fe-865b-f3de59c16981, type = RequirementUsage)
